Prefetch & Caching

In [1]:
import tensorflow as tf
import time

In [2]:
tf.__version__

'2.19.0'

In [21]:
class FileDataset(tf.data.Dataset):
    def read_files_in_batches(num_samples):
        # open files
        time.sleep(0.03)
        for sample_idx in range(num_samples):
            time.sleep(0.015)
            yield(sample_idx,)

    def __new__(cls, num_samples=3):
        print("new called")
        return tf.data.Dataset.from_generator(
            cls.read_files_in_batches,
            output_signature=tf.TensorSpec(shape=(1,), dtype=tf.int64),
            args=(num_samples,)
        )





In [22]:
# artificial delay
def benchmark(dataset, num_epochs=2):
# def train(dataset, num_epochs=2):
    for epoch_num in range(num_epochs):
        for sample in dataset:
            time.sleep(0.001)

def benchmark(dataset, num_epochs=2):
    for epoch_num in range(num_epochs):
        for sample in dataset:
            # Performing a training step
            time.sleep(0.01)

In [23]:
%%timeit
benchmark(FileDataset())

new called
new called
new called
new called
new called
new called
new called
new called
393 ms ± 56.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [26]:
%%timeit
benchmark(FileDataset().prefetch(1))

new called
new called
new called
new called
new called
new called
new called
new called
547 ms ± 147 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
%%timeit
benchmark(FileDataset().prefetch(tf.data.AUTOTUNE))

new called
new called
new called
new called
new called
new called
new called
new called
526 ms ± 174 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Cache API

In [28]:
dataset = tf.data.Dataset.range(5)
for d in dataset:
    print(d.numpy())

0
1
2
3
4


In [29]:
dataset = dataset.map(lambda x: x**2)
for d in dataset:
    print(d.numpy())

0
1
4
9
16


In [30]:
dataset = dataset.cache()
list(dataset.as_numpy_iterator())

[0, 1, 4, 9, 16]

In [31]:
list(dataset.as_numpy_iterator())

[0, 1, 4, 9, 16]

In [32]:
# creating dummy map fn. with some kind of dealay
def mapped_function(s):
    tf.py_function(lambda: time.sleep(0.03), [],())
    return s

In [34]:
%%timeit -n1 -r1
benchmark(FileDataset().map(mapped_function), 5)

new called
1.62 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [35]:
%%timeit -n1 -r1
# less time with cache fn.
benchmark(FileDataset().map(mapped_function).cache(), 5)

new called
834 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
